# Gestion des données tabulaires

Les données tabulaires (ensemble de données regroupées par colonnes et par lignes) sont universelles et omniprésentes. On les retrouve aussi bien dans nos listes de courses, nos listes de taches, la gestion comptable et le suivi des processus.

Des outils dédiés leurs sont consacrés (les tableurs) et elles sont à la base également de solutions élaborées comme par exemple pour le stockage (les 'tables' des bases de données relationnelles) et pour le traitement (les 'dataFrame' Pandas) de gros volumes de données.

Pourtant, malgré cette importante popularité, il n'existe que très peu d'outils permettant de comprendre, d'analyser et d'exploiter la structuration de ce type de données.

L'objet de cet article est donc de vous présenter les apports de ce type d'approche.

L'article est découpé en deux parties:
- une [première partie](#1---Structures-tabulaires) explicative accompagnée d'un exemple,
- une [deuxième partie](#2---Exemple-d'outil) justificative s'appuyant sur un outil dédié

Les liens présents dans les deux parties facilitent l'interactivité entre celles-ci.

# 1 - Structures tabulaires
## Que trouve-t-on dans un tableau ?

La première structure identifiable est évidente : les lignes et les colonnes !
Néanmoins, ces deux notions ne sont pas équivalentes, les colonnes représentent la 'sémantique' des données et les lignes représentent les objets rangés suivant cette structuration définie par les colonnes.

Si l'on observe maintenant comment sont utilisés les tableaux, on peut identifier trois usages principaux :
- la classification : Il s'agit de regrouper les objets par catégories afin de pouvoir par exemple en faire une exploitation statistique,
- le croisement : Il consiste à représenter toutes les combinaisons entre deux paramètres comme par exemple dans les représentations matricielles
- la caractérisation : Elle correspond à la documentation de propriétés définies.

> *Prenons un exemple :*
>
>|id|produit|aliment	|quantité	|prix|validité|disponibilité|
|:------:|:------:|:------:|:------:|:------:|:------:|:------:|
|11|pomme|fruit	|sachet 1 kg	|1	|du 1/7/2022 au 31/12/2022|oui|
|12|pomme|fruit	|carton 10 kg	|9|du 1/7/2022 au 31/12/2022|oui|
|13|orange|fruit	|sachet 1 kg|2|du 1/7/2022 au 31/12/2022|fin 2022|
|14|orange|fruit	|carton 10 kg	|18	|du 1/7/2022 au 31/12/2022|fin 2022|
|15|piment|légume	|sachet 1 kg	|1.5|du 1/7/2022 au 31/12/2022|oui|
|16|piment|légume	|carton 10 kg|13|du 1/7/2022 au 31/12/2022|fin 2022|
|17|banane|fruit	|sachet 1 kg	|0.5|du 1/7/2022 au 31/12/2022|oui|
|18|banane|fruit	|carton 10 kg	|4|du 1/7/2022 au 31/12/2022|oui|
>
>*Il s'agit d'une liste de prix de différents aliments en fonction d'un conditionnement pour l'année 2022.*
>
> *On retrouve ici :*
> - *la classification : entre les produits et les aliments,* 
> - *le croisement : entre les produits et les quantités,*
> - *la caractérisation : la disponibilité du produit*

On peut noter que les deux usages de classification et de croisement conduisent à dupliquer fortement les informations avec les risques d'erreurs associés.

## Analyse des colonnes

Les relations entre deux colonnes peuvent être qualifiées et mesurées en fonction de ces trois usages. 
Ceci permet par exemple :
- de contrôler qu'une exigence est bien respectée,
- de retrouver les données qui ne respectent pas l'exigence 

> *Dans l'exemple précédent, on vérifie que le champ 'aliment' est bien 'derivé' du champ 'produit' ([voir partie 2](#lien-entre-deux-colonnes)).*
>
> *Si l'on introduit une association ('banane', 'légume') au lieu de ('banane', 'fruit'), l'exigence de couplage n'est plus respectée [(voir partie 2)](#erreur-de-saisie). Les données incohérentes peuvent alors être explicitée [(voir partie 2)](#détection-des-erreurs).*


## Analyse globale

L'analyse de l'ensemble des liens entre champs permet de représenter la structure globale du tableau.    
Une première représentation exploite le lien de classification. Elle se traduit par un arbre de dépendance entre colonnes.     
Une deuxième représentation qualifie chacun des champs en fonction de son rôle dans la structure globale.

> *La première représentation appliquée à l'exemple se traduit par l'arborescence suivante ([voir partie 2](#représentation-globale)) :*
> ```python
> -1: root (8)    
>      0 : id (8)    
>            1 : produit (4)    
>                  2 : aliment (2)    
>            3 : quantite (2)
>            4 : prix (8)
>            6 : disponibilite (2)
>      5 : validite (1)
> ```
> *La racine de cette arborescence (root) représente la liste des lignes du tableaux.*    
> *Le champ 'aliment' est bien 'dérivé' du champ 'produit'.*    
> *la valeur entre parenthèse représente le nombre de valeurs différentes (les valeurs les plus faibles sont situées dans les feuilles de l'arbre).*
>     
> *La deuxième représentation est la suivante ([voir partie 2](#représentation-globale)) :*
> ```python
> {'id': 'variable',
> 'produit': 'primary',
> 'aliment': 'secondary',
> 'quantite': 'primary',
> 'prix': 'coupled',
> 'validite': 'unique',
> 'disponibilite': 'variable'}
> ```
> *On distingue les catégories suivantes :*
> - *primary: qualifie les champs 'croisés' ('produit' et 'quantite')*
> - *secondary: représente les champs 'dérivés' des champs 'primary' ('aliment')*
> - *coupled: est une catégorie spécique indiquant que chaque valeur de ce champ est associée à une unique valeur d'un autre champ (chaque valeur du champ 'prix' correspond à une valeur du champ 'id')*
> - *unique: pour les champs composés d'une seule valeur ('validite')
> - *variable: pour les autres champs qui ne disposent pas de relations particulières ('id', 'disponibilite')*

## Export vers des outils d'analyse
Certains outils exploitent nativement ces structures de données. C'est le cas par exemple de l'outil Xarray qui représente les données à la fois sous forme matricielle et sous forme de dépendances avec chacuns des axes de la matrice.

> *L'export des données de l'exemple vers Xarray produit la structure suivante ([voir partie 2](#Export-Xarray))  :*
> ```python
><xarray.DataArray 'prix' (produit: 4, quantite: 2)>
>array([[20, 2],
>       [15, 1.5],
>       [5, 0.5],
>       [10, 1]], dtype=object)
>Coordinates:
>  * produit   (produit) object 'orange' 'piment' 'banane' 'pomme'
>  * quantite  (quantite) object 'carton 10 kg' 'sachet 1 kg'
>    aliment   (produit) object 'fruit' 'legume' 'fruit' 'fruit'
>Attributes:
>    validite:  du 1/7/2022 au 31/12/2022
> ```
> *Les données dans la matrice sont celles liées au champ 'prix', on aurait pu également choisir celles d'un autre champ 'variable' ('id' ou 'disponibilite')*
> *Les deux dimensions de la matrice correspondent aux deux champs 'produit' et 'quantite'.*
> *Le champ 'aliment' est associé au champ 'produit' au niveau de la structure 'coordinates'.*
> *Le champ 'annee' est unique et se trouve donc associé à la structure 'attributes'.*

## Optimisation des formats d'échange
Le format privilégié d'échange des données tabulaires est le format csv qui est l'image exacte d'un tableau (chaque ligne du tableau est une ligne du fichier csv).   
Ce format n'est pas optimal dans la mesure où les données dupliquées du tableau se retrouve également dupliquées dans le fichier csv.   
    
La prise en compte des liens entre champs permet de supprimer toutes les duplications et de les remplacer par une codification optimisée. Le gain de volume est alors conséquent puisque la taille du fichier est divisée par 5 à 10 sur des jeux de données importants.

> *Les données ci-dessous représentent au format JSON les données de l'exemple dans une structure équivalente à celle d'un fichier csv ([voir partie 2](#Format-d'échange)) :*      
>
>```python
>[['produit', ['piment', 'piment', 'banane', 'banane', 'pomme', 'pomme', 'orange', 'orange']],
 ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
 ['aliment', ['legume', 'legume', 'fruit', 'fruit', 'fruit', 'fruit', 'fruit', 'fruit']],
 ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
 ['id', [15, 16, 17, 18, 11, 12, 13, 14]],
 ['disponibilite', ['oui', 'fin 2022', 'oui', 'oui', 'oui', 'oui', 'fin 2022', 'fin 2022']],
 ['prix', [1.5, 15, 0.5, 5, 1, 10, 2, 20]]]
>
>```
> *Si maintenant, on représente les mêmes informations mais sans duplication et avec une codification optimisée, on obtient un format JSON beaucoup plus simple ([voir partie 2](#Format-d'échange)) :*      
>```python
> [['produit', ['piment', 'banane', 'pomme', 'orange']],
> ['quantite', ['sachet 1 kg', 'carton 10 kg']],
> ['aliment', ['fruit', 'legume'], [0, [1, 0, 0, 0]]],
> ['validite', ['du 1/7/2022 au 31/12/2022']],
> ['id', [15, 16, 17, 18, 11, 12, 13, 14]],
> ['disponibilite', ['fin 2022', 'oui'], [4, [1, 0, 1, 1, 1, 1, 0, 0]]],
> ['prix', [1.5, 15, 0.5, 5, 1, 10, 2, 20], 4]]
>```
> *Ce formt JSON peut également être partagé dans un format binaire (standard CBOR) avec un gain supplémentaire de volume de données.*

## Identifiant unique

Disposer d'un identifiant unique pour une structure de données est un pré-requis pour garantir l'intégrité d'un jeu de données et assurer la traçabilité des modifications.
Cette notion est complexe et ne peut être traitée au niveau de la représentation textuelle ou physique d'une donnée. Elle doit nécessairement s'appliquer au niveau sémantique de la donnée.

>*Par exemple, les entiers 011, 11 et 1,1 10e1 doivent être considérés comme identiques tout comme deux dates doivent être considérées comme identiques si leur valeur dans le référentiel UTC sont identiques.*

Pour une structure de données tabulaire, l'identifiant doit également intégrer (en plus de l'identification de chacune des données qu'il contient) l'identifiant de la structure tabulaire qui porte les données.

>*Par exemple, le lien entre une données et la ligne ou la colonne qui la contient doit être pris en compte contrairement à l'ordre des colonnes qui n'est pas significatif.*

>*Ainsi, dans l'exemple présenté, les données reconstruites à partir du fichier JSON complet ou bien à partir du fichier JSON optimisé ou encore à partir du fichier binaire présentent le même identifiant.*


------------------

# 2 - Exemple d'outil

### lien entre deux colonnes
(voir la partie 1 : [analyse des colonnes](#Analyse-des-colonnes)

In [1]:
from observation import Ilist
#création de l'exemple
tarif = Ilist.obj([['id', [11, 12, 13, 14, 15, 16, 17, 18]],
                   ['produit', ['pomme', 'pomme', 'orange', 'orange', 'piment', 'piment', 'banane', 'banane']],
                   ['aliment', ['fruit', 'fruit', 'fruit', 'fruit', 'legume', 'legume', 'fruit', 'fruit']],
                   ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
                   ['prix', [1, 10, 2, 20, 1.5, 15, 0.5, 5]],
                   ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
                   ['disponibilite', ['oui', 'oui','fin 2022','fin 2022','oui','fin 2022','oui','oui']]])

id, produit, aliment, quantite, prix, validite, dispo = tarif.lindex
#test du lien entre le champ 'aliment' et le champ 'produit'
aliment.isderived(produit)

True

### erreur de saisie
(voir la partie 1 : [analyse des colonnes](#Analyse-des-colonnes)

In [2]:
tarif_erreur = Ilist.obj([['id', [11, 12, 13, 14, 15, 16, 17, 18]],
                   ['produit', ['pomme', 'pomme', 'orange', 'orange', 'piment', 'piment', 'banane', 'banane']],
                   ['aliment', ['fruit', 'fruit', 'fruit', 'fruit', 'legume', 'legume', 'legume', 'fruit']],
                   ['quantite', ['sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg', 'sachet 1 kg', 'carton 10 kg']],
                   ['prix', [1, 10, 2, 20, 1.5, 15, 0.5, 5]],
                   ['validite', ['du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022', 'du 1/7/2022 au 31/12/2022']],
                   ['disponibilite', ['oui', 'oui','fin 2022','fin 2022','oui','fin 2022','oui','oui']]])

id_, produit_, aliment_, quantite_, prix_, validite_, dispo_ = tarif_erreur.lindex

aliment_.isderived(produit_)

False

### détection des erreurs
(voir la partie 1 : [analyse des colonnes](#Analyse-des-colonnes)

In [3]:
produit_.coupling(aliment_)
tarif_erreur.getduplicates(['produit'], indexview=['produit', 'aliment'] )

[('banane', 'legume'), ('banane', 'fruit')]

### représentation globale
(voir la partie 1 : [analyse globale](#Analyse-globale)

In [4]:
print(tarif.tree)
tarif.category

-1: root (8)
      0 : id (8)
            1 : produit (4)
                  2 : aliment (2)
            3 : quantite (2)
            4 : prix (8)
            6 : disponibilite (2)
      5 : validite (1)


{'id': 'variable',
 'produit': 'primary',
 'aliment': 'secondary',
 'quantite': 'primary',
 'prix': 'coupled',
 'validite': 'unique',
 'disponibilite': 'variable'}

### Export Xarray

In [5]:
tarif.to_xarray(varname='prix')

<xarray.DataArray 'prix' (produit: 4, quantite: 2)>
array([[1.5, 15],
       [0.5, 5],
       [1, 10],
       [2, 20]], dtype=object)
Coordinates:
  * produit   (produit) object 'piment' 'banane' 'pomme' 'orange'
  * quantite  (quantite) object 'sachet 1 kg' 'carton 10 kg'
    aliment   (produit) object 'legume' 'fruit' 'fruit' 'fruit'
Attributes:
    validite:  du 1/7/2022 au 31/12/2022

In [6]:
tarif.to_xarray(varname='disponibilite')

<xarray.DataArray 'disponibilite' (produit: 4, quantite: 2)>
array([['oui', 'fin 2022'],
       ['oui', 'oui'],
       ['oui', 'oui'],
       ['fin 2022', 'fin 2022']], dtype=object)
Coordinates:
  * produit   (produit) object 'piment' 'banane' 'pomme' 'orange'
  * quantite  (quantite) object 'sachet 1 kg' 'carton 10 kg'
    aliment   (produit) object 'legume' 'fruit' 'fruit' 'fruit'
Attributes:
    validite:  du 1/7/2022 au 31/12/2022

### Format d'échange

In [ ]:
from pprint import pprint
tarif.setcanonorder()
fichier_csv = tarif.json(modecodec='full', encoded=True)
fichier_opt = tarif.json(modecodec='optimize', encoded=True)
fichier_bin = tarif.json(modecodec='optimize', encoded=True, encode_format='cbor')
print('taille : ', len(fichier_csv), len(fichier_opt), len(fichier_bin), '\n')
print('\nmode équivalent csv :')
pprint(tarif.json(modecodec='full'))
print('\nmode optimisé :')
pprint(tarif.json(modecodec='optimize'))

### identifiant

In [ ]:
print(hash(Ilist.from_obj(fichier_csv)))
print(hash(Ilist.from_obj(fichier_opt)))
print(hash(Ilist.from_obj(fichier_bin)))